# Simulation Basics

In this notebook, we'll introduce you to the basics of running a simulation of a manipulator arm in Colab. Specifically, we'll use a simulation of the [Kinova Gen3](https://www.kinovarobotics.com/en/products/gen3-robot) that we have in the lab. We've tried to make the simulation interface as close as possible to the hardware interface, so it's relatively easy to apply tools you've developed in simulation to the real robot. 

# Notebook Setup

Our simulation depends on a bunch of software (most notably [Drake](https://drake.mit.edu/)) that isn't installed by default on Google's servers. The following cell installs this software, which is pretty complex, so it will probably take a minute or two. You'll need to run a similar cell every time you use the simulator. Fortunately, once a cloud machine has been provisioned and the software intalled, it should remain availible to you for 12 hours or so. 

In [1]:
#@title Run Notebook Setup
import importlib
import sys
import os
from urllib.request import urlretrieve
import subprocess
import shutil

assert 'google.colab' in sys.modules, "This notebook is meant to be run in google colab!"

drake_url = "https://drake-packages.csail.mit.edu/tmp/drake-0.27.0-pip-bionic.tar.gz"
if importlib.util.find_spec('pydrake') is None:
    # We're in colab and don't have pydrake, so install it on the cloud machine.
    if os.path.isdir('/opt/drake'):
        shutil.rmtree('/opt/drake')
    print("Installing Drake")
    urlretrieve(drake_url, 'drake.tar.gz')
    subprocess.run(['mkdir', '/opt/drake'])
    subprocess.run(['tar', '-xzf', 'drake.tar.gz', '-C', '/opt/drake'], check=True)
    
    print("Installing other dependencies")
    subprocess.run(["pip3", "install", "meshcat"])
    subprocess.run(['apt-get', 'update', '-o', 'APT::Acquire::Retries=4', '-qq'], check=True)
    with open("/opt/drake/share/drake/setup/packages-bionic.txt", "r") as f:
        packages = f.read().splitlines()
    subprocess.run(['apt-get', 'install', '-o',
                    'APT::Acquire::Retries=4', '-o', 'Dpkg::Use-Pty=0',
                    '-qy', '--no-install-recommends'] + packages,
                    check=True)
    
    v = sys.version_info
    path = f"/opt/drake/lib/python{v.major}.{v.minor}/site-packages"
    if importlib.util.find_spec('pydrake') is None:
        sys.path.append(path)

# Start a meshcat server
print("Starting Meshcat")
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=['--ngrok_http_tunnel'])

# Clone our github repo
install_path = '/opt/kinova_drake'
if not os.path.isdir(install_path):
    print("Cloning github repo")
    subprocess.run(['git','clone','https://github.com/vincekurtz/kinova_drake.git',install_path])
sys.path.append(install_path)

# Install open3d point cloud library
print("Installing Open3D")
subprocess.run(['pip3','install','open3d'])

print("Done!")

Installing Drake
Installing other dependencies
Starting Meshcat
Cloning github repo
Installing Open3D
Done!


You don't need to know the details of how the setup cell above works, but if you want to check them out you can just double click the cell. 

# Viewing the Simulator

One of the things that the setup code above did was define a variable called `web_url` which is a web address where you can view the simulator. The easiest way to do this is probably to open the URL in a new tab. For now it should be just a blank environment with a grid, a blue background, and nothing else. 

In [ ]:
web_url

Alternatively, you can embed the `web_url` in an html `iframe`, which allows you to view the simulator in this page, using the code below.

In [ ]:
import IPython
width=800
height=400
if web_url[:5] != 'https':
    web_url = 'https' + web_url[4:]
iframe = '<iframe src=' + web_url + ' width=' + str(width) + ' height=' + str(height) + '></iframe>'
IPython.display.HTML(iframe)

# Running a Simulation

Now for the fun part, where we set up and run a simulation of our Kinova Gen3 robot! 

First we'll set up a `KinovaStation` object. This object provides our basic interface with the robot and other objects in the simulated environment. The `KinovaStation` is defined in [this package](https://github.com/vincekurtz/kinova_drake), which we installed as part of the notebook setup. Feel free to take a look at the souce code if you're interested. 

Specifically, we'll set up a scenario in which we have the robot, a simple gripper, and a single peg. We'll use a timestep of 0.002 seconds to run the simulation. You can learn more about the `KinovaStation` class by looking at the souce code [here](https://github.com/vincekurtz/kinova_drake/blob/master/kinova_station/simulation_station.py).

In [11]:
from kinova_station import KinovaStation

station = KinovaStation(time_step=0.002)
station.SetupSinglePegScenario()
station.ConnectToMeshcatVisualizer(start_server=False)
station.Finalize()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://2378e0d5f344.ngrok.io/static/
Connected to meshcat-server.


Now we'll create a simple controller. This simple controller is defined by a command sequence which we call `cs`. This command sequence is basically a list of `Command` objects, which define a target pose (position and orientation), the amount of time this target should be sent to the robot for, and whether the gripper should be open or closed. 

The controller also has two parameters `Kp` and `Kd`, which are proportional and derivative gains for the low-level controller. 

The controller class is defined [here](https://github.com/vincekurtz/kinova_drake/blob/master/controllers/command_sequence_controller.py) if you want to read more about it. 



In [12]:
import numpy as np
from controllers import CommandSequenceController, CommandSequence, Command
from kinova_station import EndEffectorTarget

cs = CommandSequence([])
cs.append(Command(
    name="move_one",
    target_pose=np.array([0.5*np.pi, 0.0, 0.5*np.pi, 0.5, 0.0, 0.4]),
    duration=4,
    gripper_closed=False))
cs.append(Command(
    name="move_two",
    target_pose=np.array([0.5*np.pi, 0.0, 0.3*np.pi, 0.5, 0.2, 0.7]),
    duration=4,
    gripper_closed=True))

Kp = 10*np.eye(6)
Kd = 2*np.sqrt(Kp)

controller = CommandSequenceController(
    cs, 
    Kp=Kp,
    Kd=Kd)

Then we connect the controller and the station in one system diagram, so that commands from the controller are sent to the robot and data from the robot is sent to the controller. 

In [13]:
from pydrake.all import * 

builder = DiagramBuilder()
station = builder.AddSystem(station)
controller = builder.AddSystem(controller)
controller.ConnectToStation(builder, station)

diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

Finally, we set some initial positions for the robot and a simulated peg, and initialize the simulation. 

In [14]:
station.go_home(diagram, diagram_context, name="Home")
station.SetManipulandStartPositions(diagram, diagram_context)

simulator = Simulator(diagram, diagram_context)
simulator.set_target_realtime_rate(1.0)
simulator.set_publish_every_time_step(False)

And run the simulation!

In [ ]:
simulator.Initialize()
simulator.AdvanceTo(10.0)

# Challenges

- Modify the controller so that the robot picks up the block, moves it to a new location, and drops it.
- Find an online introduction to PD/PID controllers. What are the parameters `Kp` and `Kd` doing? What happens when you change them?
- Try different values for the `time_step`. What happens when the timestep is very large or very small?
- Break the simulation in at least 2 different ways (not syntax/programming errors). Why does what you do break the simulation?
- Try moving the target position outside the robot's workspace. What happens? Research "inverse kinematics" and why they might fail in certain situations. 